### Import the dependencies

In [1]:
import nltk
import string
import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.layers import Dense, Dropout, Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import MeanSquaredError, CategoricalCrossentropy
from tensorflow.sparse import  SparseTensor, to_dense

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.preprocessing import StandardScaler,OneHotEncoder

import gc

### Run garbage collection to free up memory

In [2]:
gc.collect()

20

### Increase the size of the columns to be able to view more data

In [3]:
# Set display column width to 500

pd.set_option('display.max_colwidth', 500)

### Read the training, validation, and test data

In [4]:
# Read the csv files

data_train = pd.read_csv("train.txt", sep=';', header=None)
data_val = pd.read_csv("val.txt", sep=';', header=None)
data_test = pd.read_csv("test.txt", sep=';', header=None)

# Set the column headers of the dataframes

data_train.columns = ['text', 'label']
data_val.columns = ['text', 'label']
data_test.columns = ['text', 'label']

# Combine all three dataframes into one

data = pd.concat((data_train,data_val,data_test),axis=0)

data.head()

,text,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplace i will know that it is still on the property,love
4,i am feeling grouchy,anger


### Remove punctuation

In [5]:
### Create a punct_count column to contain the number of punctuation characters in each line of data
data["punct_count"] = len([char for char in data["text"] if char in string.punctuation])

### Get the frequency of the number of punctuation counts 
data["punct_count"].value_counts()

0    20000
Name: punct_count, dtype: int64

### Tokenization

In [6]:
# nltk stopwords
from nltk.tokenize import word_tokenize

# Additional Stop words
data['text_tokenized'] = data["text"].apply(lambda x: word_tokenize(x))

In [7]:
data.head()

,text,label,punct_count,text_tokenized
0,i didnt feel humiliated,sadness,0,"[i, didnt, feel, humiliated]"
1,i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake,sadness,0,"[i, can, go, from, feeling, so, hopeless, to, so, damned, hopeful, just, from, being, around, someone, who, cares, and, is, awake]"
2,im grabbing a minute to post i feel greedy wrong,anger,0,"[im, grabbing, a, minute, to, post, i, feel, greedy, wrong]"
3,i am ever feeling nostalgic about the fireplace i will know that it is still on the property,love,0,"[i, am, ever, feeling, nostalgic, about, the, fireplace, i, will, know, that, it, is, still, on, the, property]"
4,i am feeling grouchy,anger,0,"[i, am, feeling, grouchy]"


In [8]:
data["label"].value_counts()

joy         6761
sadness     5797
anger       2709
fear        2373
love        1641
surprise     719
Name: label, dtype: int64

### Remove stopwords

In [9]:
# nltk stopwords
stopwords = nltk.corpus.stopwords.words('english')

# Additional Stop words
stop_words = ["arent", "cant", "couldnt", "didnt", "doesnt", "dont", "hadnt", "hasnt", "havent", "hed", "hell", "hes", "Id", "Ill", "Im", "Ive", "isnt", "lets", "mightnt", "mustnt", "shant", "shed", "shell", "shes", "shouldnt", "thats", "theres", "theyd", "theyll", "theyre", "theyve", "wed", "were", "weve", "werent", "whatll", "whatre", "whats", "whatve", "wheres", "whos", "wholl", "whore", "whos", "whove", "wont", "wouldnt", "youd", "youll", "youre", "youve"]

In [10]:
def remove_stopwords(word_list):
    return " ".join([WordNetLemmatizer().lemmatize(word) for word in word_list if word not in stopwords and word not in stop_words])

data["text_tokenized_nostop"] = data["text_tokenized"].apply(lambda x: remove_stopwords(x))
data.head()

,text,label,punct_count,text_tokenized,text_tokenized_nostop
0,i didnt feel humiliated,sadness,0,"[i, didnt, feel, humiliated]",feel humiliated
1,i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake,sadness,0,"[i, can, go, from, feeling, so, hopeless, to, so, damned, hopeful, just, from, being, around, someone, who, cares, and, is, awake]",go feeling hopeless damned hopeful around someone care awake
2,im grabbing a minute to post i feel greedy wrong,anger,0,"[im, grabbing, a, minute, to, post, i, feel, greedy, wrong]",im grabbing minute post feel greedy wrong
3,i am ever feeling nostalgic about the fireplace i will know that it is still on the property,love,0,"[i, am, ever, feeling, nostalgic, about, the, fireplace, i, will, know, that, it, is, still, on, the, property]",ever feeling nostalgic fireplace know still property
4,i am feeling grouchy,anger,0,"[i, am, feeling, grouchy]",feeling grouchy


### Vectorize

### Run garbage collection to free up memory

In [11]:
gc.collect()

20

In [14]:
tf_vect = TfidfVectorizer(use_idf=False)
X_tf_vect = tf_vect.fit_transform(data.iloc[:data.shape[0],:]["text_tokenized_nostop"])

X_features = pd.DataFrame(X_tf_vect.toarray())
X_features.head()

,0,1,2,3,4,5,6,7,8,9,...,15037,15038,15039,15040,15041,15042,15043,15044,15045,15046
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Convert the features dataframe, X_features, to a numpy array and store the result in X

In [15]:
X = X_features.to_numpy()
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Encode the labels column into 6 one-hot-encoded columns, one column for each of the available labels

In [16]:
encoder = OneHotEncoder()
y = encoder.fit_transform(data[["label"]])
pd.DataFrame(y.toarray())

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
19995,1.0,0.0,0.0,0.0,0.0,0.0
19996,1.0,0.0,0.0,0.0,0.0,0.0
19997,0.0,0.0,1.0,0.0,0.0,0.0
19998,0.0,0.0,1.0,0.0,0.0,0.0


### Convert sparse matrix to SparseTensor

In [17]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return SparseTensor(indices, coo.data, coo.shape)

### Retrieve the training, validation, and test datasets from the larger dataset

In [18]:
# Training
X_train = X[:data_train.shape[0],:]
y_train = y[:data_train.shape[0]]

# Validation
X_val = X[data_train.shape[0]:data_train.shape[0] + data_val.shape[0],:]
y_val = y[data_train.shape[0]:data_train.shape[0] + data_val.shape[0]]

# Test
X_test = X[data_train.shape[0] + data_val.shape[0]:data_train.shape[0]
              + data_val.shape[0] 
              + data_test.shape[0],:]
y_test = y[data_train.shape[0] + data_val.shape[0]:data_train.shape[0]
              + data_val.shape[0] 
              + data_test.shape[0]:]


### Convert target dataframes to dense tensors

In [19]:
y_train = to_dense(convert_sparse_matrix_to_sparse_tensor(y_train))
y_val = to_dense(convert_sparse_matrix_to_sparse_tensor(y_val))
y_test = to_dense(convert_sparse_matrix_to_sparse_tensor(y_test))

### Create the neural network

In [20]:
seq = Sequential()
seq.add(Dense(units=1000,activation="relu",input_dim=X_train.shape[1]))
seq.add(Dropout(0.2))
seq.add(Dense(units=50,activation="relu"))
seq.add(Dropout(0.2))
seq.add(Dense(units=6,activation="softmax"))

seq.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

### Run garbage collection to free up memory

In [21]:
gc.collect()

20

### Fit the sequential model to the training, validating with the validation data

In [22]:
# Train the model

fit_model = seq.fit(X_train,y_train,epochs=20, validation_data=(X_val, y_val))

Epoch 1/20
500/500 [==============================] - 62s 125ms/step - loss: 0.8267 - accuracy: 0.7140 - val_loss: 0.3602 - val_accuracy: 0.8895
Epoch 2/20
500/500 [==============================] - 62s 124ms/step - loss: 0.2463 - accuracy: 0.9167 - val_loss: 0.2947 - val_accuracy: 0.8970
Epoch 3/20
500/500 [==============================] - 63s 125ms/step - loss: 0.1760 - accuracy: 0.9417 - val_loss: 0.2984 - val_accuracy: 0.9020
Epoch 4/20
500/500 [==============================] - 63s 126ms/step - loss: 0.1395 - accuracy: 0.9519 - val_loss: 0.2958 - val_accuracy: 0.9045
Epoch 5/20
500/500 [==============================] - 63s 126ms/step - loss: 0.1178 - accuracy: 0.9628 - val_loss: 0.3221 - val_accuracy: 0.9020
Epoch 6/20
500/500 [==============================] - 64s 128ms/step - loss: 0.1014 - accuracy: 0.9682 - val_loss: 0.3383 - val_accuracy: 0.9030
Epoch 7/20
500/500 [==============================] - 64s 128ms/step - loss: 0.0887 - accuracy: 0.9722 - val_loss: 0.3425 - val_ac

### Run garbage collection to free up memory

In [23]:
gc.collect()

1502

### Predict the labels of the test dataset

In [24]:
y_test_pred = seq.predict(X_test,use_multiprocessing=True)

### Retrieve the test portion of the larger dataframe 

In [25]:
y_test_nparray = y[data_train.shape[0] + data_val.shape[0]:data_train.shape[0]
              + data_val.shape[0] 
              + data_test.shape[0]:]

### Function to convert lists of floats into lists of ints

In [26]:
def float_to_int(fl):
    int_2d = []
    for row in fl:
        i_row = []
        for x in row:
            i_row.append(int(x+0.5))
        int_2d.append(i_row)
    
    return np.array(int_2d)
    

### Display the raw predictions

In [27]:
y_test_pred

array([[4.8819135e-07, 9.6991259e-09, 3.0806346e-05, 1.2461024e-07,
        9.9996853e-01, 5.2724734e-11],
       [5.6741206e-05, 7.8038801e-07, 4.8891685e-05, 2.4712463e-07,
        9.9989331e-01, 1.4269165e-09],
       [5.9076105e-11, 1.5371188e-13, 2.5840925e-09, 9.4133228e-16,
        1.0000000e+00, 7.2693840e-18],
       ...,
       [1.6293231e-11, 1.9279285e-11, 9.9999952e-01, 4.9657132e-07,
        5.9977440e-11, 6.8748857e-15],
       [5.3603660e-11, 1.7351228e-14, 9.9978906e-01, 2.1092226e-04,
        3.1640371e-11, 1.2624303e-15],
       [1.5846966e-04, 9.5732957e-01, 2.5813063e-04, 2.1751407e-07,
        1.3380712e-06, 4.2252332e-02]], dtype=float32)

### Convert the elements of the prediction dataframe to int

In [28]:
y_test_pred = float_to_int(y_test_pred)
y_test_pred

array([[0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       ...,
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]])

### Determine the accuracy of the predicted results

In [30]:
(y_test_pred == y_test_nparray).sum()/(y_test_pred.shape[0] * y_test_pred.shape[1])

0.962